# Raw data exploration

### Imports

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

from src.utils.const import DATA_DIR

In [2]:
ROOT_DIR = os.path.join(os.getcwd(), '..')
RAW_DIR = os.path.join(ROOT_DIR, DATA_DIR, 'raw')
FIGURE_DIR = os.path.join(ROOT_DIR, 'reports', 'figures')

### Useful path to data

## Generic useful function

In [3]:
from src.utils.wrapper import drop, drop_na, rename, reset_index, fill_na, convert_to, extract_stat_feature

### movies.csv

In [4]:
movies = pd.read_csv(
    os.path.join(RAW_DIR, 'movies.csv'),
    encoding='utf-8',
    dtype={'movieId':'int32', 'title': 'string', 'genres': 'category'}
)

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   movieId  58098 non-null  int32   
 1   title    58098 non-null  string  
 2   genres   58098 non-null  category
dtypes: category(1), int32(1), string(1)
memory usage: 871.8 KB


#### Specific movies functions used in pipe

In [5]:
def extract_year_from_title(df: pd.DataFrame) -> pd.DataFrame:
    regex = '.*\\((\\d{4})\\).*'
    df['year'] = df['title'].str.extract(pat=regex, expand=False)
    return df


def extract_title_length(df: pd.DataFrame) -> pd.DataFrame:
    df['title_length'] = df['title'].str.len()
    return df


def encode_genre(df: pd.DataFrame) -> pd.DataFrame:
    genres = df['genres'].str.split('|')
    mlb = MultiLabelBinarizer()
    encoded_genre = pd.DataFrame(
        mlb.fit_transform(genres),
        index=df['movieId'],
        columns=mlb.classes_
    )
    df = pd.merge(df, encoded_genre, on='movieId', how='inner')
    return df


def remove_no_genres(df: pd.DataFrame) -> pd.DataFrame:
    df_no_genre = df[df['(no genres listed)'] == 1].index
    df.drop(index=df_no_genre, inplace=True)
    return df

#### Pipe movies

In [6]:
movies = movies. \
    pipe(extract_year_from_title). \
    pipe(convert_to, 'year', 'float32'). \
    pipe(fill_na, 'year', 'median'). \
    pipe(extract_title_length). \
    pipe(convert_to, 'title_length', 'int32'). \
    pipe(encode_genre). \
    pipe(remove_no_genres). \
    pipe(drop, ['title', 'genres', '(no genres listed)'])

movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53832 entries, 0 to 58097
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       53832 non-null  int32  
 1   year          53832 non-null  float32
 2   title_length  53832 non-null  int32  
 3   Action        53832 non-null  int32  
 4   Adventure     53832 non-null  int32  
 5   Animation     53832 non-null  int32  
 6   Children      53832 non-null  int32  
 7   Comedy        53832 non-null  int32  
 8   Crime         53832 non-null  int32  
 9   Documentary   53832 non-null  int32  
 10  Drama         53832 non-null  int32  
 11  Fantasy       53832 non-null  int32  
 12  Film-Noir     53832 non-null  int32  
 13  Horror        53832 non-null  int32  
 14  IMAX          53832 non-null  int32  
 15  Musical       53832 non-null  int32  
 16  Mystery       53832 non-null  int32  
 17  Romance       53832 non-null  int32  
 18  Sci-Fi        53832 non-nu

### tags.csv

In [7]:
tags = pd.read_csv(
    os.path.join(RAW_DIR, 'tags.csv'),
    encoding='utf-8',
    usecols=['movieId', 'tag'],
    dtype={'movieId':'int32', 'tag': 'string'}
)

tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   movieId  1108997 non-null  int32 
 1   tag      1108981 non-null  string
dtypes: int32(1), string(1)
memory usage: 12.7 MB


#### Pipe tags

In [8]:
tags = tags. \
    pipe(drop_na). \
    pipe(extract_stat_feature, ['movieId'], 'tag', ['count']). \
    pipe(reset_index). \
    pipe(rename, {'count':'tag_count'}). \
    pipe(convert_to, 'tag_count', 'int32')

tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45981 entries, 0 to 45980
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   movieId    45981 non-null  int64
 1   tag_count  45981 non-null  int32
dtypes: int32(1), int64(1)
memory usage: 539.0 KB


### ratings.csv

In [9]:
ratings = pd.read_csv(
    os.path.join(RAW_DIR, 'ratings.csv'),
    encoding='utf-8',
    usecols=['movieId', 'rating'],
    dtype={'movieId':'int32', 'rating':'float32'}
)

ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   movieId  int32  
 1   rating   float32
dtypes: float32(1), int32(1)
memory usage: 211.7 MB


#### Pipe ratings

In [10]:
ratings = ratings. \
    pipe(extract_stat_feature, ['movieId'], 'rating', ['count', 'mean']). \
    pipe(reset_index). \
    pipe(rename, {'count':'rating_count', 'mean':'rating_mean'}). \
    pipe(convert_to, 'rating_count', 'int32')

ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53889 entries, 0 to 53888
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       53889 non-null  int64  
 1   rating_count  53889 non-null  int32  
 2   rating_mean   53889 non-null  float32
dtypes: float32(1), int32(1), int64(1)
memory usage: 842.1 KB
